In [ ]:
import subprocess
import os
from joblib import Parallel, delayed
import glob
from concurrent.futures import ProcessPoolExecutor, as_completed  # 切换到 ProcessPoolExecutor 以支持实时进度

def load_environment(env_file):
    cmd = f'bash -c "source {env_file} && env"'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    new_env = {}
    for line in result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            if not key.startswith('BASH_FUNC_'):
                new_env[key] = value
    
    os.environ.update(new_env)
    return os.environ.copy()

def run_colm(run_path, nml_path, log_path, nml_name, updated_env):
    """
    处理单个 nml 文件，返回成功/失败状态（不打印进度，由主脚本处理）。
    """
    nml_file = f'{nml_path}{nml_name}.nml'
    log_file = f'{log_path}{nml_name}.txt'
    
    try:
        # 用 'w' 模式打开文件，重置内容
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== 处理 {nml_name}.nml ===\n")
            log.flush()
            
            commands = [
                [f'{run_path}mksrfdata.x', nml_file],
                [f'{run_path}mkinidata.x', nml_file],
                [f'{run_path}colm.x', nml_file]
            ]
            
            for cmd in commands:
                log.write(f"执行命令: {' '.join(cmd)}\n")
                log.flush()
                
                subprocess.run(cmd, 
                               env=updated_env, 
                               stdout=log, 
                               stderr=subprocess.STDOUT, 
                               text=True)
                
                log.write("\n" + "="*50 + "\n")
                log.flush()
            
            log.write(f"=== {nml_name} 处理完成 ===\n")
            log.flush()
        
        return True  # 成功
    except Exception as e:
        # 如果失败，也记录到日志
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== {nml_name}.nml 处理失败: {str(e)} ===\n")
        return False

if __name__ == "__main__":
    forcing ='FLUXNET-CH4'
    mode ='no_spin_up'

    env_file = '/share/home/dq089/soft/gnu-env'
    run_path = '/share/home/dq076/mode/ME/CoLM202X_CH4_s/run/'

    nml_path = f'{run_path}site/{forcing}/{mode}/'
    log_path = f'{nml_path}logs/'  
    os.makedirs(log_path, exist_ok=True)

    updated_env = load_environment(env_file)
    nml_files = glob.glob(f'{nml_path}*.nml')
    nml_names = [os.path.splitext(os.path.basename(nml_file))[0] for nml_file in nml_files]
    print(f"发现 {len(nml_files)} 个 .nml 文件：{nml_names}")
    
    # 切换到 ProcessPoolExecutor 以支持 as_completed 实时进度
    max_workers = min(24, os.cpu_count() or 1)
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务，返回 future 对象
        future_to_nml = {
            executor.submit(run_colm, run_path, nml_path, log_path, nml_name, updated_env): nml_name
            for nml_name in nml_names
        }
        
        # 维护剩余任务集合
        remaining_nml = set(nml_names)
        completed_count = 0
        
        # 实时监控完成
        for future in as_completed(future_to_nml):
            nml_name = future_to_nml[future]
            try:
                success = future.result()
                if success:
                    completed_count += 1
                remaining_nml.discard(nml_name)  # 移除已完成（无论成功/失败）
                
                # 打印进度：总数 + 剩余列表
                print(f"=== {nml_name} 处理完成（成功: {success}) ===")
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                else:
                    print("所有任务已完成！")
                print("-" * 50)
                
            except Exception as exc:
                print(f"{nml_name} 执行异常: {exc}")
                remaining_nml.discard(nml_name)
                completed_count += 1  # 视作完成（失败）
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                print("-" * 50)
    
    print("批量处理结束。")

In [3]:
import subprocess
import os
from joblib import Parallel, delayed
import glob
from concurrent.futures import ProcessPoolExecutor, as_completed  # 切换到 ProcessPoolExecutor 以支持实时进度

def load_environment(env_file):
    cmd = f'bash -c "source {env_file} && env"'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    new_env = {}
    for line in result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            if not key.startswith('BASH_FUNC_'):
                new_env[key] = value
    
    os.environ.update(new_env)
    return os.environ.copy()

def run_colm(run_path, nml_path, log_path, nml_name, updated_env):
    """
    处理单个 nml 文件，返回成功/失败状态（不打印进度，由主脚本处理）。
    """
    nml_file = f'{nml_path}{nml_name}.nml'
    log_file = f'{log_path}{nml_name}.txt'
    
    try:
        # 用 'w' 模式打开文件，重置内容
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== 处理 {nml_name}.nml ===\n")
            log.flush()
            
            commands = [
                # [f'{run_path}mksrfdata.x', nml_file],
                # [f'{run_path}mkinidata.x', nml_file],
                [f'{run_path}colm.x', nml_file]
            ]
            
            for cmd in commands:
                log.write(f"执行命令: {' '.join(cmd)}\n")
                log.flush()
                
                subprocess.run(cmd, 
                               env=updated_env, 
                               stdout=log, 
                               stderr=subprocess.STDOUT, 
                               text=True)
                
                log.write("\n" + "="*50 + "\n")
                log.flush()
            
            log.write(f"=== {nml_name} 处理完成 ===\n")
            log.flush()
        
        return True  # 成功
    except Exception as e:
        # 如果失败，也记录到日志
        with open(log_file, 'w', encoding='utf-8') as log:
            log.write(f"=== {nml_name}.nml 处理失败: {str(e)} ===\n")
        return False

if __name__ == "__main__":
    forcing ='FLUXNET-CH4'
    mode ='spin_up_80yr'

    env_file = '/share/home/dq089/soft/gnu-env'
    run_path = '/share/home/dq076/mode/ME/CoLM202X_CH4_s/run/'

    nml_path = f'{run_path}site/{forcing}/{mode}/'
    log_path = f'{nml_path}logs/'  
    os.makedirs(log_path, exist_ok=True)

    updated_env = load_environment(env_file)
    nml_files = glob.glob(f'{nml_path}*.nml')
    nml_names = [os.path.splitext(os.path.basename(nml_file))[0] for nml_file in nml_files]
    print(f"发现 {len(nml_files)} 个 .nml 文件：{nml_names}")
    
    # 切换到 ProcessPoolExecutor 以支持 as_completed 实时进度
    max_workers = min(24, os.cpu_count() or 1)
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务，返回 future 对象
        future_to_nml = {
            executor.submit(run_colm, run_path, nml_path, log_path, nml_name, updated_env): nml_name
            for nml_name in nml_names
        }
        
        # 维护剩余任务集合
        remaining_nml = set(nml_names)
        completed_count = 0
        
        # 实时监控完成
        for future in as_completed(future_to_nml):
            nml_name = future_to_nml[future]
            try:
                success = future.result()
                if success:
                    completed_count += 1
                remaining_nml.discard(nml_name)  # 移除已完成（无论成功/失败）
                
                # 打印进度：总数 + 剩余列表
                print(f"=== {nml_name} 处理完成（成功: {success}) ===")
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                else:
                    print("所有任务已完成！")
                print("-" * 50)
                
            except Exception as exc:
                print(f"{nml_name} 执行异常: {exc}")
                remaining_nml.discard(nml_name)
                completed_count += 1  # 视作完成（失败）
                print(f"已完成总数: {completed_count}/{len(nml_names)}")
                if remaining_nml:
                    print(f"剩余未处理: {sorted(list(remaining_nml))}")
                print("-" * 50)
    
    print("批量处理结束。")

发现 79 个 .nml 文件：['US-Tw1', 'US-Tw4', 'US-Uaf', 'ID-Pag', 'US-MRM', 'HK-MPM', 'CH-Cha', 'US-HRA', 'US-ICs', 'FI-Lom', 'US-EML', 'KR-CRK', 'IT-Cas', 'CH-Dav', 'FI-Si2', 'US-LA2', 'US-LA1', 'US-Tw5', 'JP-Mse', 'US-Myb', 'AT-Neu', 'US-NGC', 'US-EDN', 'MY-MLM', 'RU-Ch2', 'DE-Dgw', 'US-ORv', 'DE-SfN', 'US-BZS', 'US-CRT', 'US-A10', 'US-StJ', 'US-Sne', 'US-DPW', 'CA-SCB', 'RU-Cok', 'PH-RiF', 'FR-LGt', 'US-NGB', 'CA-SCC', 'US-Snd', 'RU-Che', 'NZ-Kop', 'US-Bi2', 'BW-Gum', 'US-Srr', 'US-Tw3', 'US-Ho1', 'BR-Npw', 'JP-SwL', 'JP-BBY', 'US-Pfa', 'US-BZF', 'DE-Hte', 'FI-Hyy', 'US-A03', 'US-Los', 'US-Bi1', 'US-Ivo', 'US-Bes', 'RU-Fy2', 'US-MAC', 'US-Twt', 'US-WPT', 'US-BZB', 'US-HRC', 'NL-Hor', 'CH-Oe2', 'DE-Zrk', 'CN-Hgu', 'US-Beo', 'US-NC4', 'BW-Nxr', 'IT-BCi', 'SE-Deg', 'US-OWC', 'FI-Sii', 'US-Atq', 'UK-LBT']
=== ID-Pag 处理完成（成功: True) ===
已完成总数: 1/79
剩余未处理: ['AT-Neu', 'BR-Npw', 'BW-Gum', 'BW-Nxr', 'CA-SCB', 'CA-SCC', 'CH-Cha', 'CH-Dav', 'CH-Oe2', 'CN-Hgu', 'DE-Dgw', 'DE-Hte', 'DE-SfN', 'DE-Zrk', 'FI

Process ForkProcess-55:
Process ForkProcess-64:
Process ForkProcess-53:
Process ForkProcess-62:
Process ForkProcess-51:
Process ForkProcess-58:
Process ForkProcess-56:
Process ForkProcess-72:
Process ForkProcess-49:
Process ForkProcess-63:
Process ForkProcess-68:
Process ForkProcess-61:
Process ForkProcess-71:
Process ForkProcess-52:
Process ForkProcess-67:
Process ForkProcess-65:
Process ForkProcess-59:
Process ForkProcess-57:
Process ForkProcess-66:
Process ForkProcess-60:
Process ForkProcess-50:
Traceback (most recent call last):
  File "/share/home/dq076/software/miniconda3/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/home/dq076/software/miniconda3/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **sel

KeyboardInterrupt: 